In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
root_dir = ""
data_dir = "../data/"
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=False)
    root_dir = "/content/gdrive/My Drive/gen/"
    data_dir = root_dir + "data/"
except:
    pass

In [ ]:
import re
import csv
import pandas as pd
import numpy as np
from csv import writer
import random
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain


davinci = OpenAI(model_name='text-davinci-003', temperature=1, max_tokens=3000)

In [ ]:
def gen_topics(num_topics=50, max_runs=2, save_output_file_path=None):
    curr_run = 0
    generate_topics = f"Generate {num_topics} topics useful for inter-table tasks."
    topics = davinci(generate_topics)
    curr_run += 1
    post_process_topics = re.split('\d+.', topics.replace("\n", " "))
    final_topics = []
    for topic in post_process_topics:
        topic = topic.strip()
        if topic:
            final_topics.append(topic)
    while len(set(final_topics)) <= num_topics:
        required_num = num_topics - len(set(final_topics))
        previous_topics = ",".join(final_topics)
        if curr_run < max_runs:
            generate_new_topics = f"Here are a set of topics: {previous_topics}. Generate {required_num} topics useful for inter-table tasks that are different from the previous topics."
            newer_topics = davinci(generate_new_topics)
            curr_run += 1
            post_process_topics = re.split('\d+.', newer_topics.replace("\n", " "))
            for topic in post_process_topics:
                topic = topic.strip()
                if topic:
                    final_topics.append(topic)
                else:
                    break
    if save_output_file_path is not None:
        f = open(save_output_file_path,'w')
        f.write("Topics\n")
        for t in final_topics:
            f.write(t+"\n")
            f.close()
    return final_topics[:num_topics]

In [ ]:
num_topics=50
output_file_path = data_dir + f"topics_{num_topics}.txt"
# if topic file is present, we can just read that txt file
topics = []
if os.path.isfile(output_file_path):
    f = open(output_file_path,'r')
    lines = f.readlines()
    topics = [t.strip() for t in lines[1:num_topics+1]]
else:
    topics = gen_topics(save_output_file_path=output_file_path)

In [ ]:
print("Topics", topics)
print("Topics len", len(topics))

In [ ]:
from numpy.core.multiarray import empty
import random
import string

def ran_gen(size, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for x in range(size))


def gen_table(task, task_label, table_1, table_2, table_separation="pipe", save_output_dir=None, save_ground_truth=None):
    is_success_task = "cannot" if not task_label else "can"
    separator_char = ","
    if table_separation == "pipe":
        separator_char = "|"
    prompt = f"Create 2 tables with cells separated by {separator_char}. "
    # Table 1 Characteristics
    curr_topic_1 = table_1['topic']
    curr_rows_1 = table_1['rows']
    curr_columns_1 = table_1['columns']
    curr_textuality_1 = table_1['textuality'] * curr_rows_1 * curr_columns_1
    prompt += f"Table 1 has {curr_columns_1} columns, {curr_rows_1} rows, and {curr_textuality_1} words, related to {curr_topic_1}. "
    # Table 2 Characteristics
    curr_topic_2 = table_2['topic']
    curr_rows_2 = table_2['rows']
    curr_columns_2 = table_2['columns']
    curr_textuality_2 = table_2['textuality'] * curr_rows_2 * curr_columns_2
    prompt += f"Table 2 has {curr_columns_2} columns, {curr_rows_2} rows, and {curr_textuality_2} words, related to {curr_topic_2}. "
    # Topics for both tables
    topics = [curr_topic_1, curr_topic_2]
    # Set task label in the prompt
    if task_label:
        match_column_num = random.randint(2, 5)
        prompt += f"They can be {task}ed because they have only {match_column_num} semantically common columns and at least 1 related same row values across the tables. "
    else:
        prompt += f"They cannot be {task}ed because they have 0 semantically common columns. "
    # Instruct format of answering the task
    prompt += "Answer the above task in the following format:\n"
    prompt += "Table 1: {table 1}\nTable 2: {table 2}\n"
    if task_label:
        prompt += "\nKey: {key column in table 1}\n"
    if "table" in table_1:
        table_1_table = table_1["table"]
        prompt += f"Table 1: {table_1_table}"
    else:
        prompt += "Table 1:"
    prompt_output = davinci(prompt)
    table_inds_to_process = []
    if "table" in table_1:
        if task_label:
            match_output = re.search(r"(?<=Table 2:)(.*)(?=Key)", prompt_output, flags=re.IGNORECASE | re.MULTILINE | re.DOTALL)
        else:
            match_output = re.search(r"(?<=Table 2:)(.*)", prompt_output, flags=re.IGNORECASE | re.MULTILINE | re.DOTALL)
        if match_output:
            curr_table = match_output.group(0)
            lines = curr_table.strip().splitlines()
            cleaned_table = "\n".join([line for line in lines if '|' in line])
            table_2['table'] = cleaned_table
            table_inds_to_process.append(1)
        else:
            return prompt, prompt_output, None, None, None
    else:
        match_output = re.search(r"(.*)(?=Table 2)", prompt_output, flags=re.IGNORECASE | re.MULTILINE | re.DOTALL)
        if match_output:
            curr_table = match_output.group(0)
            lines = curr_table.strip().splitlines()
            cleaned_table = "\n".join([line for line in lines if '|' in line])
            table_1['table'] = cleaned_table
            table_inds_to_process.append(0)
        else:
            return prompt, prompt_output, None, None, None
        if task_label:
            match_output = re.search(r"(?<=Table 2:)(.*)(?=Key)", prompt_output, flags=re.IGNORECASE | re.MULTILINE | re.DOTALL)
        else:
            match_output = re.search(r"(?<=Table 2:).*", prompt_output, flags=re.IGNORECASE | re.MULTILINE | re.DOTALL)
        if match_output:
            curr_table = match_output.group(0)
            lines = curr_table.strip().splitlines()
            cleaned_table = "\n".join([line for line in lines if '|' in line])
            table_2['table'] = cleaned_table
            table_inds_to_process.append(1)
        else:
            return prompt, prompt_output, None, None, None
  
    # Get groundtruth
    table_groundtruth = ""
    actual_table_index = 0
    if task_label:
        match_output = re.search(r"(?<=Key:).*\n?", prompt_output, flags=re.IGNORECASE | re.MULTILINE | re.DOTALL)
        if match_output:
            table_groundtruth = match_output.group(0)
        else:
            table_groundtruth = "N/A"
    for t in table_inds_to_process:
        curr_table = None
        if t == 0:
            curr_table = table_1
        elif t == 1:
            curr_table = table_2
        table_rows = curr_table['table'].split("\n")
        if save_output_dir is not None:
            if 'table_csv' in curr_table:
                continue
            ran_gen_id = ran_gen(8)
            output_file_gen = f"{topics[t]}_{ran_gen_id}.csv"
            output_file_path = os.path.join(save_output_dir, output_file_gen)
            if t == 0:
                table_1['table_csv'] = output_file_gen
            elif t == 1:
                table_2['table_csv'] = output_file_gen
            with open(output_file_path, 'a+') as csv_f:
                writer_csv_f = writer(csv_f, delimiter='|')
                for t_rows in range(0, len(table_rows)):
                    if len(table_rows[t_rows]) >= 2:
                        writer_csv_f.writerow(table_rows[t_rows].split('|'))
    if save_ground_truth is not None:
        with open(save_ground_truth, 'a+') as gtf:
            writer_gtf = writer(gtf)
            task_gt_label = 0
            if task_label:
                task_gt_label = 1
            writer_gtf.writerow([table_1['table_csv'], table_2['table_csv'], task_gt_label, table_groundtruth])
    return prompt, prompt_output, table_1, table_2, table_groundtruth

In [ ]:
random.seed(42)
def generate_sparse_table(input_dir, table_filepath, sparsity, columns_subset=None):   
    input_csv = input_dir + "/" + table_filepath
    if not os.path.exists(input_csv):
        print("Could not find file")
        return
    with open(input_csv, 'r') as input_csv_file:
        input_table = []
        csv_reader = csv.reader(input_csv_file, delimiter='|')
        line_count = 0
        for row in csv_reader:
            input_table.append(row)
        if columns_subset == None:
            num_columns = len(input_table[0]) - 1
            columns_subset = [i for i in range(num_columns)]
        num_rows = len(input_table) - 1
        num_null_cells = int(sparsity * num_rows * len(columns_subset))
        # Generate a list of all cell positions in the table
        all_positions = []
        for i in range(num_rows):
            for j in range(len(columns_subset)):
                curr_row = input_table[i]
                if 0 <= j < len(curr_row):
                    all_positions.append((i,j))
        # Randomly select positions to set as null (None) values
        null_positions = random.sample(all_positions, num_null_cells)
        # Create the table with null values
        for i, row in enumerate(input_table):
            for j, value in enumerate(row):
                if (i, j) in null_positions:
                    input_table[i][j] = None
        #table = [[None if (i, j) in null_positions else input_table[i][j] for j in range(len(columns_subset))] for i in range(num_rows)]
        ran_gen_id = ran_gen(8)
        table_initial = table_filepath.split('.')[0]
        output_file_gen = f"{table_initial}_{ran_gen_id}.csv"
        return input_table, output_file_gen


def gen_sparse_union_benchmark(sparsity, benchmark_src_dir, benchmark_tgt_dir, groundtruth_src_csv, groundtruth_tgt_csv,continuation=False):
    if groundtruth_tgt_csv is not None:
        with open(groundtruth_tgt_csv, 'a+') as gtf:
            writer_gtf = writer(gtf)
            if not continuation:
                writer_gtf.writerow(["query_table", "data_lake_table", "unionable", "intent_col_name"])
            gtf.flush()
    with open(groundtruth_src_csv, 'r') as src_gtf:
        csv_reader = csv.reader(src_gtf, delimiter=',')
        next(csv_reader)
        query_tables = {}
        datalake_tables = {}
        for row in csv_reader:
            query_table, datalake_table, unionable, intent_col = row
            
            if query_table not in query_tables:
                query_sparse_table, query_sparse_table_name = generate_sparse_table(benchmark_src_dir + "/query", query_table, sparsity)
                query_tables[query_table] = [query_sparse_table, query_sparse_table_name]
            if datalake_table not in datalake_tables:
                datalake_sparse_table, datalake_sparse_table_name = generate_sparse_table(benchmark_src_dir + "/datalake", datalake_table, sparsity)
                datalake_tables[datalake_table] = [datalake_sparse_table, datalake_sparse_table_name]
                
            query_info = query_tables[query_table]
            datalake_info = datalake_tables[datalake_table]
            
            query_sparse_full_path = os.path.join(benchmark_tgt_dir + "/query", query_info[1])
            datalake_sparse_full_path = os.path.join(benchmark_tgt_dir + "/datalake", datalake_info[1])
            
            if not os.path.exists(query_sparse_full_path):
                with open(query_sparse_full_path, 'a+') as csv_f:
                    writer_csv_f = writer(csv_f, delimiter='|')
                    curr_table = query_info[0]
                    for t_rows in range(0, len(curr_table)):
                        writer_csv_f.writerow(curr_table[t_rows])

            if not os.path.exists(datalake_sparse_full_path):
                with open(datalake_sparse_full_path, 'a+') as csv_f:
                    writer_csv_f = writer(csv_f, delimiter='|')
                    curr_table = datalake_info[0]
                    for t_rows in range(0, len(curr_table)):
                        writer_csv_f.writerow(curr_table[t_rows])
            
            with open(groundtruth_tgt_csv, 'a+') as gtf:
                writer_gtf = writer(gtf)
                writer_gtf.writerow([query_info[1], datalake_info[1], unionable, intent_col])
                
    
benchmark_src_dir = '../ugen_v1'
benchmark_tgt_dir = '../ugen_v1_sparse_20'
groundtruth_src_csv = f'{benchmark_src_dir}/groundtruth.csv'
groundtruth_tgt_csv = f'{benchmark_tgt_dir}/groundtruth.csv'
sparsity=0.20

if not os.path.exists(benchmark_tgt_dir):
    os.makedirs(benchmark_tgt_dir)
    os.makedirs(benchmark_tgt_dir + "/query")
    os.makedirs(benchmark_tgt_dir + "/datalake")
     
gen_sparse_union_benchmark(sparsity, benchmark_src_dir, benchmark_tgt_dir, groundtruth_src_csv, groundtruth_tgt_csv)

In [ ]:
def reset_table(topic):
    table = {}
    table['topic'] = topic
    table['rows'] = random.randint(5, 20)
    table['columns'] = random.randint(10, 15)
    table['sparsity'] = random.uniform(0, 1)
    table['textuality'] = random.randint(1, 5)
    return table

def gen_union_benchmark(log_file, benchmark_dir, groundtruth_csv, topics, data_lake_size=12, continuation=False):
    with open(data_dir + 'log.txt', 'a+') as f:
        if not os.path.exists(benchmark_dir):
            os.makedirs(benchmark_dir)
        output_dir = benchmark_dir + "data"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        groundtruth_csv = benchmark_dir + "groundtruth.csv"
        with open(groundtruth_csv, 'a+') as gtf:
            writer_gtf = writer(gtf)
            if not continuation:
                writer_gtf.writerow(["query_table", "data_lake_table", "unionable", "intent_col_name"])
            gtf.flush()
        curr_topic_counter = 48
        curr_range_counter = 0
        while curr_topic_counter < len(topics):
            print("At the first while loop: ", curr_topic_counter)
            topic_index = curr_topic_counter
            task = "union"
            task_label = True
            topic = topics[topic_index]
            print(topic)
            table_1 = reset_table(topic)
            table_2 = reset_table(topic)
            prompt, tables, table_1, table_2_copy, table_groundtruth = gen_table(task, task_label, table_1, table_2,
                                                                                 table_separation="pipe",
                                                                                 save_output_dir=output_dir,
                                                                                 save_ground_truth=groundtruth_csv)
            f.write(f"\nprompt: {prompt}\n")
            f.write(f"\ntables:\n")
            f.write(tables)
            f.flush()
            if table_1 is None:
                continue
            curr_range_counter = 1
            while curr_range_counter <= (data_lake_size - 1):
                i = curr_range_counter
                if i % 2 == 0:
                    task_label = True
                else:
                    task_label = False
                prompt, tables, table_1_copy, table_2_copy, table_groundtruth = gen_table(task, task_label, table_1, reset_table(topic),
                                                                                          table_separation="pipe",
                                                                                          save_output_dir=output_dir,
                                                                                          save_ground_truth=groundtruth_csv)
                f.write(f"\nprompt: {prompt}\n")
                f.write(f"\ntables:\n")
                f.write(tables)
                f.flush()
                if table_1_copy is not None:
                    curr_range_counter += 1
                    print("curr_range_counter: ", curr_range_counter)
                if (table_1 is not None) and (curr_range_counter == data_lake_size):
                    curr_topic_counter += 1
                    curr_range_counter = 0
                    print("curr_topic_counter: ", curr_topic_counter)
                    break
    return

In [ ]:
log_file = data_dir + 'log.txt'
benchmark_dir = root_dir + "ugen_v1/"
os.makedirs(benchmark_dir, exist_ok=True)
groundtruth_csv = benchmark_dir + "groundtruth.csv"
gen_union_benchmark(log_file, benchmark_dir, groundtruth_csv, topics, data_lake_size=20, continuation=True)